# Attention Visualization and Experiment Tracking
注意力可视化和实验跟踪

##### Tracking with wandb

* https://github.com/wandb/wandb

## Preparation for Google Colab

In [1]:
import os
from google.colab import drive

#  挂载 google 云盘
drive.mount("/content/drive")

print(os.getcwd())  # /content

# print(os.listdir("/content/drive/MyDrive/"))

# print(os.listdir("/content/drive/MyDrive/Colab Notebooks"))

# if os.getcwd() != "/content/drive/MyDrive":
#     os.chdir("/content/drive/MyDrive")

# print(os.getcwd())

Mounted at /content/drive
/content


In [ ]:
# 提前将 requirements.txt 放在 google 云盘上
!pip install -r /content/drive/MyDrive/requirements.txt

In [3]:
subdir = "ch11c"
work_path = "/content/drive/MyDrive/" + subdir
if not os.path.exists(work_path):
    os.mkdir(work_path)
os.chdir(work_path)
print(os.getcwd())

/content/drive/MyDrive/ch11c


In [4]:
!apt-get install tree && tree -a "./"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 23 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 0s (659 kB/s)
Selecting previously unselected package tree.
(Reading database ... 124947 files and directories currently installed.)
Preparing to unpack .../tree_2.0.2-1_amd64.deb ...
Unpacking tree (2.0.2-1) ...
Setting up tree (2.0.2-1) ...
Processing triggers for man-db (2.10.2-1) ...
./

0 directories, 0 files


In [5]:
!pip freeze > colab-requirements-11c.txt

In [7]:
import wandb
wandb.login(relogin=True)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hzhilamp (hzhilamp-lab) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [8]:
from transformers.trainer_utils import set_seed

set_seed(42)

In [9]:
from torch import cuda

device = "cuda" if cuda.is_available() else "cpu"
device

'cuda'

## Loading pre-trained model and tokenizer

In [10]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification

model_path = "distilbert-base-uncased"
tokenizer = DistilBertTokenizerFast.from_pretrained(model_path)
model = DistilBertForSequenceClassification.from_pretrained(
    model_path, id2label={0: "NEG", 1: "POS"}, label2id={"NEG": 0, "POS": 1}
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Loading popular IMDB dataset

In [11]:
from datasets import load_dataset

# to take entire dataset from original train 25 K AND TEST 25K
"""
imdb_train= load_dataset('imdb', split="train")
imdb_test= load_dataset('imdb', split="test[:6250]+test[-6250:]")
imdb_val= load_dataset('imdb', split="test[6250:12500]+test[-12500:-6250]")

"""
# to take smaller portion 4K for train, 1K for test and 1K for validation
imdb_train = load_dataset("imdb", split="train[:2000]+train[-2000:]")
imdb_test = load_dataset("imdb", split="test[:500]+test[-500:]")
imdb_val = load_dataset("imdb", split="test[500:1000]+test[-1000:-500]")

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [12]:
imdb_train.shape, imdb_test.shape, imdb_val.shape

((4000, 2), (1000, 2), (1000, 2))

In [13]:
enc_train = imdb_train.map(
    lambda e: tokenizer(e["text"], padding=True, truncation=True),
    batched=True,
    batch_size=1000,
)
enc_test = imdb_test.map(
    lambda e: tokenizer(e["text"], padding=True, truncation=True),
    batched=True,
    batch_size=1000,
)
enc_val = imdb_val.map(
    lambda e: tokenizer(e["text"], padding=True, truncation=True),
    batched=True,
    batch_size=1000,
)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [14]:
enc_train.shape, enc_test.shape, enc_val.shape

((4000, 4), (1000, 4), (1000, 4))

In [15]:
import pandas as pd

pd.DataFrame(enc_train)

,text,label,input_ids,attention_mask
0,I rented I AM CURIOUS-YELLOW from my video sto...,0,"[101, 1045, 12524, 1045, 2572, 8025, 1011, 375...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,"""I Am Curious: Yellow"" is a risible and preten...",0,"[101, 1000, 1045, 2572, 8025, 1024, 3756, 1000...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,If only to avoid making this type of film in t...,0,"[101, 2065, 2069, 2000, 4468, 2437, 2023, 2828...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,This film was probably inspired by Godard's Ma...,0,"[101, 2023, 2143, 2001, 2763, 4427, 2011, 2643...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,"Oh, brother...after hearing about this ridicul...",0,"[101, 2821, 1010, 2567, 1012, 1012, 1012, 2044...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...
3995,A hit at the time but now better categorised a...,1,"[101, 1037, 2718, 2012, 1996, 2051, 2021, 2085...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3996,I love this movie like no other. Another time ...,1,"[101, 1045, 2293, 2023, 3185, 2066, 2053, 2060...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3997,This film and it's sequel Barry Mckenzie holds...,1,"[101, 2023, 2143, 1998, 2009, 1005, 1055, 8297...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3998,'The Adventures Of Barry McKenzie' started lif...,1,"[101, 1005, 1996, 7357, 1997, 6287, 18506, 100...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


## Preparing training settings with TrainingArguments and training with Trainer class

TrainingArguments is the subset of the arguments we use in our example scripts which relate to the training loop itself.

In [16]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    # The output directory where the model predictions and checkpoints will be written
    output_dir="./MyIMDBModel",
    do_train=True,
    do_eval=True,
    #  The number of epochs, defaults to 3.0
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    # Number of steps used for a linear warmup
    warmup_steps=100,
    weight_decay=0.01,
    logging_strategy="steps",
    # TensorBoard log directory
    logging_dir="./logs",
    logging_steps=50,
    # other options : no, steps
    evaluation_strategy="steps",
    save_strategy="steps",
    fp16=True,
    load_best_model_at_end=True,
    learning_rate=5e-5,
    report_to="wandb",
    run_name="IMDB-batch-16-lr-5e-5",
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Let's design our evaluation metric as follows:

In [17]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="macro"
    )
    acc = accuracy_score(labels, preds)
    return {"Accuracy": acc, "F1": f1, "Precision": precision, "Recall": recall}

In [18]:
trainer = Trainer(
    # the pre-trained model that will be fine-tuned
    model=model,
    # training arguments that we defined above
    args=training_args,
    # training and validation dataset
    train_dataset=enc_train,
    eval_dataset=enc_val,
    compute_metrics=compute_metrics,
)

Trainer class is an optimized tool for Transformer to organize training and evalulation process for PyTorch and Tensorflow.

In [19]:
results = trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,0.680700,0.635361,0.624000,0.564361,0.774094,0.624000
100,0.415300,0.411267,0.834000,0.831408,0.855889,0.834000
150,0.328700,0.336254,0.867000,0.866596,0.871495,0.867000
200,0.321300,0.283147,0.886000,0.885978,0.886303,0.886000
250,0.304200,0.277107,0.882000,0.881863,0.883775,0.882000
300,0.175300,0.351960,0.888000,0.887821,0.890499,0.888000
350,0.160500,0.292135,0.901000,0.901000,0.901002,0.901000
400,0.177300,0.348867,0.894000,0.893877,0.895830,0.894000
450,0.198200,0.337386,0.891000,0.890798,0.893913,0.891000
500,0.166600,0.324455,0.895000,0.894934,0.895990,0.895000


Could not locate the best model at ./MyIMDBModel/checkpoint-250/pytorch_model.bin, if you are running a distributed training on multiple nodes, you should activate `--save_on_each_node`.


Finishing WandB

In [20]:
wandb.finish()

eval/Accuracy,▁▆▇████████████
eval/F1,▁▇▇████████████
eval/Precision,▁▆▆▇▇▇█████████
eval/Recall,▁▆▇████████████
eval/loss,█▄▂▁▁▂▁▂▂▂▄▃▃▄▄
eval/runtime,▁▁▇▁▂▂▃▂▂▅█▅▅▂▂
eval/samples_per_second,██▂█▇▇▆▇▇▄▁▄▄▆▇
eval/steps_per_second,██▂█▇▇▆▇▇▄▁▄▄▆▇
train/epoch,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▃▃▃▃▃▃▄▄▅▅▅▅▅▅▆▆▇▇▇▇▇▇███
train/grad_norm,▂▃▆▂▇▆▄▂▄█▁▁▁▁▁


In [21]:
!tree -a ./

./
├── colab-requirements-11c.txt
├── MyIMDBModel
│   ├── checkpoint-500
│   │   ├── config.json
│   │   ├── model.safetensors
│   │   ├── optimizer.pt
│   │   ├── rng_state.pth
│   │   ├── scheduler.pt
│   │   ├── trainer_state.json
│   │   └── training_args.bin
│   └── checkpoint-750
│       ├── config.json
│       ├── model.safetensors
│       ├── optimizer.pt
│       ├── rng_state.pth
│       ├── scheduler.pt
│       ├── trainer_state.json
│       └── training_args.bin
└── wandb
    ├── debug-internal.log -> run-20250227_075223-j964ivs1/logs/debug-internal.log
    ├── debug.log -> run-20250227_075223-j964ivs1/logs/debug.log
    ├── latest-run -> run-20250227_075223-j964ivs1
    └── run-20250227_075223-j964ivs1
        ├── files
        │   ├── config.yaml
        │   ├── output.log
        │   ├── requirements.txt
        │   ├── wandb-metadata.json
        │   └── wandb-summary.json
        ├── logs
        │   ├── debug-core.log -> /root/.cache/wandb/logs/core-debug-20250227_0748